Pozn.: zde se nacházejí importy potřebné pro to, aby po restartu kernelu běžela libovolná buňka. V rámci výkladu jsou při setkání se s určitou třídou či funkcí importy samozřejmě uvedeny.

In [ ]:
import os
import dotenv
import openai
import langchain

from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI

from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.chains import TransformChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

from langchain.chains import ConversationChain
from langchain.prompts import MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory 
from langchain.memory import ConversationTokenBufferMemory 
from langchain.memory import ConversationSummaryMemory 
from langchain.memory import ConversationEntityMemory 
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.document_loaders import BSHTMLLoader
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import Language
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents.base import Document

from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

V době psaní těchto řádků je to už několik měsíců, co vzalo ChatGPT svět útokem. Člověk se tak mohl pravidelně i ve sdělovacích prostředcích setkávat s více či méně relevantními články o AI. Tento spisek nemá ambice kráčet jim po boku - absolutně netuším, zda AI (z nějakého důvodu ztotožněná s generativními modely - např. s "doplňovači" textu ala GPT) začne nahrazovat zaměstnance, zda nás vyhubí anebo zda život půjde dál ve starých kolejích. Namísto toho bych tu chtěl ukázat, jak pracovat s modely od OpenAI a to sice jak napřímo, tak skrze framework Langchain.  
Na tomto místě musím doporučit [langchainové kurzy](https://www.deeplearning.ai/short-courses/), které jsou dostupné na deeplearning.ai a ze kterých jsem při tvoření tohoto textu vycházel.

# Obsah
- [OpenAI](#OpenAI)
  - [Tokeny](#Tokeny)
  - [API klíč](#API-klíč)
  - [Jednoduchý příklad](#Jednoduchý-příklad)
  - [Chatbot](#Chatbot)
- [Langchain](#Langchain)
  - [Šablony promptů](#Šablony-promptů)
  - [Chains](#Chains)
    - [LLMChain](#LLMChain)
    - [Sekvenční chainy](#Sekvenční-chainy)
    - [RouterChain](#RouterChain)
    - [Transformation chain](#Transformation-chain)
  - [Paměť](#Paměť)
    - [ConversationBufferMemory](#ConversationBufferMemory)
    - [ConversationBufferWindowMemory](#ConversationBufferWindowMemory)
    - [ConversationTokenBufferMemory](#ConversationTokenBufferMemory)
    - [ConversationSummaryMemory](#ConversationSummaryMemory)
    - [ConversationEntityMemory](#ConversationEntityMemory)
  - [Q&A nad dokumenty](#Q&A-nad-dokumenty)
    - [Načtení dokunetů](#Načtení-dokumentů)
      - [Načtení pdfka](#Načtení-pdfka)
      - [Načtení html souboru](#Načtení-html-souboru)
      - [Načtení webové stránky](#Načtení-webové-stránky)
    - [Splittery](#Splittery)
    - [Embeddings, vectorstore](#Embeddings,-vectorstore)
    - [Similarity search](#Similarity-search)
    - [Question answering](#Question-answering)
    - [Automatické používání metadat](#Automatické-používání-metadat)
    - [Prefix před fragmenty](#Prefix-před-fragmenty)
  - [Kompletní ukázky](#Kompletní-ukázky)
    - [Obyčejný chatbot](#Obyčejný-chatbot)
    - [Q&A chatbot](#Q&A-chatbot)

# OpenAI
### Tokeny
Pro použití OpenAI člověk napřed musí sebe i svou kartu registrovat [zde](https://platform.openai.com/). Na stránkách je poté třeba nechat si vygenerovat API klíč, s jehož pomocí při RESTovém volání OpenAI pozná, co má komu vlastně naúčtovat. Ceník nalezneme [tady](https://openai.com/pricing). Je vhodné zdůraznit, že narozdíl od dejme tomu Midjourney si člověk nekupuje měsíční předplatné, nýbrž platí za míru používání modelů. Přesněji platí za počet modelem zpracovaných tokenů. Do toho se počítají jak tokeny do modelu vstupující, tak tokeny modelem produkované. A co že je vlastně onen token? Jedná se o skupinu písmen, která tvoří slovo anebo jeho část. V angličtině v průměru vychází 1 token na 0,75 slova, v jiných jazycích je poměr horší. Pro získání reálné představy doporučuji podívat se [sem](https://platform.openai.com/tokenizer). Krom očividného omezení peněženkou je součet vstupních a výstupních tokenů (a tak i délka textu) omezen pamětí modelu (kontextem z pricing stránky) - například pro 4K GPT-3.5 model není možné mít text (otázku a odpověď) delší než 4000 tokenů - nadbytečné tokeny budou oříznuty, resp. vůbec nevzniknou. O tom se člověk může předvědčit v "klasickém" webovém GUI rozhraní ChatGPT. Model sám o sobě si nepamatuje, co se dělo v předchozím hovoru. Proto se mu musí celá historie konverzace při každé interakci posílat nanovo. Po čase je historie příliš dlouhá a tak jsou staré tokeny zahozeny a model tudíž začne zapomínat začátky konverzací.

### API klíč
První tutoriálový příklad na [stránkách openai balíčku](https://pypi.org/project/openai/) začíná s 
```python
import openai
openai.api_key = "sk-..."  # supply your API key however you choose
```
Jeho problém tkví ve skutečnosti, že je API klíč umístěn uprostřed kódu. Když si člověk nedá pozor a kód někam pošle (třeba na GitHub), může se jeho jménem (a jeho peněženkou) GPT modelů dotazovat celý internet. Proto bude lepší používat [python-dotenv](https://pypi.org/project/python-dotenv/) balíček. S jeho pomocí Python načte klíč z .env souboru a nastaví ho jako proměnnou prostředí. Jak to konkrétně provedeme? Nejprve si ve stejném adresáři, ve kterém se nalézá notebook (ipynb soubor), vytvoříme .env soubor (opravdu se takto jmenuje, tj. v názvu je jen přípona) a vložíme do něj
```
OPENAI_API_KEY=nazdar1234
```
Poté naimportujeme *dotenv* a provoláme funkci *load_dotenv*.

In [ ]:
import dotenv
dotenv.load_dotenv();

Že se proměnná prostředí načetla ověříme s pomocí balíčku *os*:

In [ ]:
import os
os.environ["OPENAI_API_KEY"]

Pokud pracujeme s Azure OpenAI, je třeba inicializovat ještě několik dalších proměnných prostředí. Na těch už nic moc tajného není, tudíž mohou být volně v kódu. Vypadají nějak takto:
```python
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = "http://testingazureopenai.openai.azure.com"
```
Api type bude asi vždy stejné, api version se bude s časem měnit a nakonec api base závisí na jméně openai resourcu. Tyto parametry se dají nejsnadněji zjistit, když v AzureAI studiu vlezeme do "Chat Playground" a v sekci "Chat Session" klepneme na "View Code". 

### Jednoduchý příklad
Nyní ale přikročme k prvnímu chatovacímu příkladu. Napřed si načteme potřebné balíčky a API klíč. Ten je potřeba vložit do *OpenAI* objektu, konkrétně do parametru *api_key*.

In [ ]:
import os
from openai import OpenAI
import dotenv

dotenv.load_dotenv()
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

Následně si vytvoříme objekt reprezentující chat a to s pomocí [*chat.completions.create*](https://platform.openai.com/docs/api-reference/chat/create). Tomu podhodíme jednak model, který chceme používat (parametr *model*), jednak dosavadní historii konverzace (parametr *messages*). Ta má podobu listu jsonů obsahujících jednak informaci o mluvčím (klíč "role" s hodnotami "user" či "assistant"), jednak samotnou promluvu (klíč "content").

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Ahoj."}
    ],
    model="gpt-3.5-turbo",
)

Výstupem je následující věc. Všimněte si, že na konci dostáváme i informaci o počtu použitých tokenů.

In [ ]:
completion

Pokud chceme ale jen odpověď chatbota, použijeme

In [ ]:
completion.choices[0].message.content

Do *create* metody lze vložit další parametry. Například *temperature*, která s rostoucí hodnotou vede k chaotičtějším/kreativnějším odpovědím. Podle dokumentace je minimum 0, maximum 2 a default 1. Obecně ale hodnoty větší než 1 vedou obvykle k nesmyslné změti písmen.

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Ahoj."}
  ],
  temperature=2
)

completion.choices[0].message.content

Do *messages* lze vložit i tzv. systémový prompt, který chatbotu říká,v jaké roli má vlastně vystupovat. Takováto věc se označuje klíčem "system".

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Jsi veverka milující oříšky. Odpovídáš maximálně ve dvou větách."},
    {"role": "user", "content": "Co bych si měl dát ke svačině?"}
  ],
  temperature=0.7
)

print(completion.choices[0].message.content)

Pozn.: výše uvedené informace platí pro verzi balíčku openai větší nebo rovnou 1.0.0 ze září 2023. Pokud musíte pracovat s verzí starší, vypadá kód nějak takto:

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Ahoj."}
  ]
)

Pokud pracujeme s Azure OpenAI, musíme napsat něco v následujícím duchu:
```python
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version=openai.api_version,
    azure_endpoint=openai.api_base,
    api_key=openai.api_key
)

completion = client.chat.completions.create(
    model="jmeno_deploymentu",
    messages=[
        {"role": "user", "content": "Ahoj."}
    ]
)
```

### Chatbot
Bohužel kód jak ho tady máme jako chatbot nefunguje - nijak se tu neukládá historie konverzací a pokaždé tak začínáme nanovo. V openai balíčku se žádná příhodná funkce ani třída nenalézá a tak si celou (byť v tomto případě ultra krátkou) logiku musíme napsat sami.

In [ ]:
chat_history = [
    {"role": "system", "content": "Jsi veverka milující oříšky. Odpovídáš maximálně ve dvou větách."}
  ]

print("Pro ukončení konverzace napište 'exit'")

while True:
    user_message = input("Uživatel: ")
    if user_message == "exit":
        break
    chat_history.append(
        {"role": "user", "content": user_message}
    )
    conversation = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=chat_history,
        temperature=0.7
    )
    chatbot_answer = conversation.choices[0].message.content
    chat_history.append(
        {"role": "assistant", "content": chatbot_answer}
    )
    print(f"Chatbot: {chatbot_answer}")

# Langchain
Sice bychom mohli pokračovat v používání čistého openai balíčku, ale museli bychom přitom programovat více, než by bylo potřeba. Existuje totiž balíček Langchain, který hromadu práce udělá za nás.  
Pozn.: I v případě Langchainu se s postupem času objevily změny. Původní kód bude dohledatelný v historii repa, které právě čtete. Krom změn kódu se objevila i potřeba nainstalovat si integrační balíček [langchain-openai](https://pypi.org/project/langchain-openai/).

### Šablony promptů

S pomocí šablon lze s minimální prací navíc přepoužívat i poměrně komplexní prompty. Pro názornost nicméně začněme něčím jednoduchým.  
Napřed si napíšeme stringový základ šablony, ve kterém vložíme dodatečné parametry do složených závorek. Vlastně to vypadá jako f-stringy, jen to f-ko na začátku chybí. Šablonu jako takovou vytvoříme s pomocí *ChatPromptTemplate.from_template*, do které jako parametr vložíme string z předchozího kroku.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template_text = """
V jedné větě shrň text obklopený trojicí uvozovek. Odpověď musí být napsána v {language} jazyce. 
Text: '''{text}'''
"""
prompt_template = ChatPromptTemplate.from_template(template_text)

Takto vypadá jádro "šablonového" objektu. V první části vidíme jména do šablony dosazovaných proměnných.

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
language = "anglickém"

Jako testovací text použijeme kousek z [wiki článku o veverkách](https://cs.wikipedia.org/wiki/Veverka_obecn%C3%A1).

In [ ]:
squirrel_text = """
Veverka obecná obvykle dorůstá 19 až 23 cm a dosahuje hmotnosti mezi 250 a 340 g někdy i víc. 
Huňatý ocas, který napomáhá udržovat rovnováhu při lezení a skocích na stromech a který veverka využívá 
jako „pokrývku“ těla při spánku, je 14,5 až 20 cm dlouhý.[3] Charakteristickým znakem veverky obecné jsou střapce 
chlupů na ušních boltcích směřující do špičky a viditelné především v zimním období. 
Stejně jako většina stromových veverek má i veverka obecná ostré a zakřivené drápy, které jí pomáhají při lezení po větvích stromů. 
"""

Proměnné se do šablony dostanou v rámci provolání metody *format_messages*. V ní by každé proměnné měl odpovídat jeden parametr nesoucí její jméno.

In [ ]:
filled_template = prompt_template.format_messages(
    language=language,
    text=squirrel_text
)

Výsledek vypadá následně:

In [ ]:
filled_template

Nyní musíme Langchainu říct, s jakým jazykovým modelem má pracovat. Pro podporované modely přitom existují separátní třídy. Seznam nalezneme [zde](https://python.langchain.com/docs/integrations/llms/). Zdůrazněme, že zatímco balíček openai dokázal obhospodařit jak OpenAI modely, tak Azure OpenAI modely, zde se jedná o separátní entity. Navíc historicky byl a asi pořád je rozdíl mezi třídami OpenAI a ChatOpenAI importovanými nyní z langchain_openai.  
Jaké parametry můžeme do konstruktoru ChatOpenAI vložit? Jedná se primárně o teplotu (parametr *temperature* s defaultní hodnotou 0.7) a jméno LLM modelu (parametr *model_name* s defaultní hodnotou "gpt-3.5-turbo"). API klíč můžeme předávat parametrem *api_key* - pokud tak explicitně neuděláme, bude Langchain klíč hledat v *os.environ\["OPENAI_API_KEY"\]*.

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo")

Když pak do metody *invoke* našeho chatovacího modelu vložíme jako parametr šablonu a výsledku se zeptáme na atribut *content*, získáme odpověď. Vidíme, že anglický jazyk sice model pochopil, ale shrnutí zabralo více než požadovanou jednu větu. no, shrnutí - text vypadá spíš jako překlad...

In [ ]:
summary_response = chat.invoke(filled_template)
print(summary_response.content)

Člověka by napadlo, že se text bez ztráty informací do jedné věty možná shrnout nedá a nízká teplota zabraňuje kreativnější práci s informacemi.

In [ ]:
chat = ChatOpenAI(
    temperature=0.7, 
    model_name="gpt-3.5-turbo"
)
summary_response = chat.invoke(filled_template)
print(summary_response.content)

Nicméně možná si model zkrátka co se instrukcí týče jenom nerozumí s češtinou tak dobře jako s angličtinou.

In [ ]:
template_text = """
Summarize the text surrounded by three quotation marks in one sentence. The answer must be written in {language} language. 
Text: '''{text}'''
"""
prompt_template = ChatPromptTemplate.from_template(template_text)
language = "czech"

filled_template = prompt_template.format_messages(
    language=language,
    text=squirrel_text
)

chat = ChatOpenAI(
    temperature=0.0, 
    model_name="gpt-3.5-turbo"
)
summary_response = chat.invoke(filled_template)
print(summary_response.content)

Možná se nám někdy stane, že bude odpověď useknutá. To nejspíš bude dáno skutečností, že defaultní hodnota parametru *max_tokens* pro *ChatOpenAI*, která má velikost 256, na celou odpověď zkrátka nestačí. Pokud nechceme být omezováni (resp. pokud chceme být omezeni jen maximální velikostí kontextového okna), vložíme do parametru -1.

In [ ]:
template_text = """
Summarize the text surrounded by three quotation marks in one sentence. The answer must be written in {language} language. 
Text: '''{text}'''
"""
prompt_template = ChatPromptTemplate.from_template(template_text)
language = "czech"

filled_template = prompt_template.format_messages(
    language=language,
    text=squirrel_text
)

chat = ChatOpenAI(
    temperature=0.0, 
    model_name="gpt-3.5-turbo",
    max_tokens=22
)
summary_response = chat.invoke(filled_template)
print(summary_response.content)

Zkusme nyní lehce komplikovanější šablonu, která se bude koukat na [popis hry Na křídlech](https://www.tlamagames.com/deskove-hry/wingspan/).

In [ ]:
wingspan_text = """
Wingspan je kompetitivní středně těžká hra využívající karty a engine building mechanismus.

Stáváte se nadšenými ornitology a sběrateli a snažíte se objevit a přilákat ty nejzajímavější ptáky da Vaší sítě rezervací. Každý pták posiluje řetěz kombinací pro daný habitat (akci). Tyto habitaty jsou zaměřeny na několik klíčových oblastí rozvoje:

    Dostat žetony jídla výběrem kostky z krmítka (dice tower)
    Kladení vajec s využitím miniaturních vajec v různých barvách
    Dobrání ze stovek unikátních karet ptáků a zahrání těchto karet

Vítězem je ten hráč, který má po 4 kolech nejvíce bodů.

Pokud máte rádi hry jako Terraforming Mars a Gizmos, tak by tahle hra měla zalétnout na Váš stůl.

Hra je určena pro 1-5 hráčů od 14 let.

Pravidla i herní materiál je v angličtině.
"""

template_text = """
From text surrounded by by three quotation marks extract following information.

1) Minimal age
2) Number of players
3) Victory condition

The answer must be written in {language} language in {format} format. 

Text: '''{text}'''
"""

prompt_template = ChatPromptTemplate.from_template(template_text)
language = "czech"
format = "json"

filled_template = prompt_template.format_messages(
    language=language,
    format=format,
    text=wingspan_text
)

chat = ChatOpenAI(
    temperature=0.0, 
    model_name="gpt-3.5-turbo"
)
summary_response = chat.invoke(filled_template)
print(summary_response.content)

In [ ]:
prompt_template = ChatPromptTemplate.from_template(template_text)
language = "english"
format = "plain text"

filled_template = prompt_template.format_messages(
    language=language,
    format=format,
    text=wingspan_text
)

chat = ChatOpenAI(
    temperature=0.0, 
    model_name="gpt-3.5-turbo"
)
summary_response = chat.invoke(filled_template)
print(summary_response.content)

V případě práce s Azure OpenAI budeme namísto *ChatOpenAI* používat *AzureChatOpenAI*. Rozdíl je i v parametrech - nebudeme specifikovat *model_name*, nýbrž *deployment_name*. V Azuru se totiž vytváří svého druhu instance modelů mimo (obvykle) v GUI, přičemž jsou pojmenovány právě jako deploymenty.  Vytvořené deploymenty člověk nalezne v Azure AI studiu v sekci "deployments" (do konstruktoru dáváme název z prvního sloupce tabulky).

```python
from langchain_openai import AzureChatOpenAI

os.environ["AZURE_OPENAI_API_KEY"] = "api_klíč"
os.environ["AZURE_OPENAI_ENDPOINT"] = "endpoint ve formátu ala http://testingazureopenai.openai.azure.com"
chat = AzureChatOpenAI(deployment_name="jmeno_deploymentu", temperature=0)
```

## Chains
Je načase přistoupit k věci, podle které nese Langchain své jméno - k chainům. Ty jsou [definovány](https://python.langchain.com/docs/modules/chains/) jako posloupnosti volání komponent, které mohou obsahovat další chainy. To by ve výsledku mělo vést k větší přehlednosti a snazší udržování kódu.
#### [LLMChain](https://python.langchain.com/docs/modules/chains/foundational/llm_chain)
Jedná se o asi nejjednodušší chain. Člověk do něj při inicializaci nasype template a jazykový model. Při samotném použití *LLMChain* provoláme s hodnotou proměnné, která se dosadí do šablony. Výsledný textový řetězec poputuje do jazykového modelu a následně obdržíme výsledek.

In [ ]:
from langchain.chains import LLMChain

chat = ChatOpenAI(temperature=0.95)
prompt_template = ChatPromptTemplate.from_template(
    "Jaké je nejvhodnější jméno pro {pet}? Odpověď má být krátka o délce jedné věty, ale měla by obsahovat zdůvodnění."
)
llm_chain = LLMChain(llm=chat, prompt=prompt_template)

In [ ]:
pet = "psa"
llm_chain.invoke(pet)

Obvykle ale nechceme slovníkovou omáčku okolo, ale jen samotnou odpověď. Kdysi bylo řešením použít metodu *run*. Ta ale brzo bude odstraněna.

In [ ]:
pet = "křečka"
llm_chain.run(pet)

Tudíž musíme explicitně "slovníkově" zmínit, co vlastně chceme vidět.

In [ ]:
pet = "králíka"
llm_chain.invoke(pet)["text"]

V případě, že má jít do šablony více proměnných, musíme tyto proměnné do metody *invoke* předávat jako slovník v parametru *input*.

In [ ]:
prompt_template_two_var = ChatPromptTemplate.from_template(
    """
    Write the best name for {pet}. Answer should be in one sentence, in {language} language, but should contain reasoning.
    """
)
llm_chain_two_var = LLMChain(llm=chat, prompt=prompt_template_two_var)
input_vars = {"pet": "cat", "language": "german"}
llm_chain_two_var.invoke(input=input_vars)

Pokud člověk chce odpovědi na více separátních vstupů, není třeba popořadě manuálně provolávat model, ale lze použít metodu *apply*. Té se podhodí list slovníků s proměnnými.

In [ ]:
input_list = [
    {"pet": "parrot", "language":"english"},
    {"pet": "bear", "language":"czech"},
    {"pet": "monkey", "language":"german"}
]

llm_chain_two_var.apply(input_list)

#### [Sekvenční chainy](https://python.langchain.com/docs/modules/chains/foundational/sequential_chains)
Sekvenční chainy jsou určeny pro situace, kdy výstup jednoho provolání modelu vkládáme jako vstup do provolání druhého. Existují ve dvou variantách:
- SimpleSequentialChain, který má jen jeden vstup a jeden výstup
- SequentialChain umožňující existenci více vstupů a výstupů

Pro ukázku SimpleSequentialChain si napřed vytvoříme dva LLMChainy.

In [ ]:
from langchain.chains import SimpleSequentialChain

chat = ChatOpenAI(temperature=0.95)

prompt_template_post = ChatPromptTemplate.from_template(
    "Write a LinkedIn post for company {company}."
)

post_chain = LLMChain(llm=chat, prompt=prompt_template_post)

In [ ]:
prompt_template_summary = ChatPromptTemplate.from_template(
    """
    Summarize the text surrounded by three quotation marks in one sentence.
    Text: '''{text}''''
    """
)

summary_chain = LLMChain(llm=chat, prompt=prompt_template_summary)

Následně s jejich pomocí vytvoříme SimpleSequentialChain.

In [ ]:
overall_chain = SimpleSequentialChain(chains=[post_chain, summary_chain], verbose=True)

Ten uvedeme do provozu provoláním metody *invoke*. Do té vložíme hodnotu, která bude dosazena do prvního LLMChainu (pořadí chainů dáno pořadím v listu vkládaného do parametru *chains*).  
Jelikož jsme *SimpleSequentialChain* vytvořili s parametrem *verbose*=True, vidíme i mezivýsledky - výstup prvního a druhého LLMChainu v modré resp. žluté barvě. A ano, jsou zde warningy. Asi se jedná o nějaké volání v pozadí; zatím jsem nepřišel na to, co mám čím nahradit.

In [ ]:
post_summary = overall_chain.invoke("Hamster food Inc.")

Výsledek jsme si uložili do proměnné *post_summary*.

In [ ]:
post_summary

S *verbose*=False žádné barevné výsledky neuvidíme.

In [ ]:
overall_chain = SimpleSequentialChain(chains=[post_chain, summary_chain], verbose=False)
post_summary = overall_chain.invoke("Hamster food Inc.")

In [ ]:
post_summary

Nyní se podívejme na *SequentialChain*. Zde první LLMChain bude mít dva vstupy. Navíc tu v parametru *output_key* říkáme, do jak pojmenované proměnné zpracované dalšími LLMChainy se má výstup tohoto chainu uložit.

In [ ]:
from langchain.chains import SequentialChain

chat = ChatOpenAI(temperature=0.95)

prompt_template_post = ChatPromptTemplate.from_template(
    "Write a LinkedIn post for company {company} with length of {sentences_count} sentences."
)

post_chain = LLMChain(llm=chat, prompt=prompt_template_post, output_key="linkendin_post")

Dva vstupy a jeden *output_key* má i druhý *LLMChain*.

In [ ]:
prompt_template_summary = ChatPromptTemplate.from_template(
    """
    Summarize the text surrounded by three quotation marks in one sentence. Use {language} language.
    Text: '''{linkendin_post}''''
    """
)

summary_chain = LLMChain(llm=chat, prompt=prompt_template_summary, output_key="summarization")

Nyní si vytvoříme *SequentialChain*. Oproti *SimpleSequentialChain* tu navíc máme parametry *input_variables* a *output_variables*. Všimněme si, že i když se proměnná *language* používá až v druhém LLMChainu, je v *input_variables*. První LLMChain ji prostě ignoruje.  
V SequentialChainu bychom mohli mít více chainů a ty by ani nemusely mít uspořádány čistě sériově. Langchain už by si je postupně pospouštěl podle potřebných input a output proměnných. Dokonce ani konec nemusí být v jednom chainu - zakončení může být více a pak je output_key každého koncového chainu umístěn do listu v *output_variables*.

In [ ]:
overall_chain = SequentialChain(
    chains=[post_chain, summary_chain],
    input_variables=["company", "sentences_count", "language"],
    output_variables=["summarization"],
    verbose=True
)

Aktivovat *SequentialChain* můžeme vložením slovníku se záznamem pro každou *input_variable* do instance chainu.

In [ ]:
overall_chain.invoke({"language":"czech", "sentences_count":3, "company":"Turtles speedtravel"})

Všimněme si ale, že i když jsme nastavili *verbose*=True, nevidíme mezivýsledky. Netušíme tak, co se vlastně uvnitř děje. Když ale nastavíme langchain do debug modu s pomocí *langchain.debug* = True, uvidíme vše do nejmenších podrobností.

In [ ]:
import langchain
langchain.debug = True

In [ ]:
overall_chain.invoke({"language":"czech", "sentences_count":3, "company":"Turtles speedtravel"})

#### RouterChain
RouterChain dokáže posoudit, na který z podřízených chainů poslat uživatelský vstup. Narozdíl od sekvenčích chainů, kde je posloupnost řízena jménem proměnných (a tedy volbou programátora), v případě LLMRouterChain se o rozzařování skutečně stará jazykový model.  
Pozn.: v této podkapitole čerpám z langchainového kurzu z deeplearning.ai opravdu extenzivně - MULTI_PROMPT_ROUTER_TEMPLATE je převzat beze změn metodou ctrl+C, ctrl+V.  
Nejprve si vytvoříme základy šablon pro programování a pro vaření.

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

it_problem_template = """You are a very smart programmer. 
You are great at answering questions about computers, programming and IT in a concise 
and easy to understand manner. 
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

cooking_template = """Your are master chef, expert at cooking and food preparation. 
When you don't know the answer to a question you admit that you don't know.


Here is a question:
{input}
"""

prompt_infos = [
    {
        "name": "programming", 
        "description": "Good for answering questions about programming and IT", 
        "prompt_template": it_problem_template
    },
    {
        "name": "food", 
        "description": "Good for answering questions about food and cooking", 
        "prompt_template": cooking_template
    }
]

Následně dojde k vytvoření instance jazykového modelu a listu slovníku o podchainech.

In [ ]:
chat = ChatOpenAI(temperature=0)

destination_chains = {}
for one_prompt_info in prompt_infos:
    name = one_prompt_info["name"]
    prompt_template = one_prompt_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=chat, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{one_prompt_info['name']}: {one_prompt_info['description']}" for one_prompt_info in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

Vytvoříme si i defaultní podchain, kam půjdou uživatelské dotazy, které se nevejdou nikam jinam.

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat, prompt=default_prompt)

Následuje extenzivní popis router šablony.

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input. 
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string - name of the prompt to use or "DEFAULT"
    "next_inputs": string - a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt 
names specified below OR it can be "DEFAULT" if the input is not
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input 
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

A vytvoření samotného routeru.

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(chat, router_prompt)

A nakonec tu máme chain, který *LLMRouterChain* obaluje a spojuje ho s podchainy.

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

Zde máme příklad programátorského podchainu.

In [ ]:
chain.invoke("What are good packages for graphs creation in Python?")

Tady je dotaz na kuchařský podchain.

In [ ]:
chain.run("How should I prepare fried cheese?")

A nakonec defaultní podchain.

In [ ]:
chain.run("Who was Joseph II?")

#### [Transformation chain](https://python.langchain.com/docs/modules/chains/foundational/transformation)
Transformation chain slouží k tomu, aby se na vstupní textový řetězec použila nějaká obecná pythoní funkce. Její jméno (tj. bez parametrů a kulatých závorek) se vloží do parametru *transofrm* konstruktoru *TransformChain*.

In [ ]:
from langchain.chains import TransformChain

def replace_animal(input_dict):
    orig_text = input_dict["text"]
    replaced_text = orig_text.replace("squirrel", "unicorn")
    replaced_text = replaced_text.replace("squirrels", "unicorns")
    replaced_text = replaced_text.replace("Squirrel", "Unicorn")
    replaced_text = replaced_text.replace("Squirrels", "Unicorns")
    return {"output_text": replaced_text}

transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=replace_animal
)

Následně se s chainy pracuje jako v předchozích příkladech.

In [ ]:
template = """Find strange formulations in following text and write why are they strange:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
chat = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo")
llm_chain = LLMChain(llm=chat, prompt=prompt)
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

In [ ]:
sequential_chain.invoke(
    """
    Squirrels are members of the family Sciuridae (/sɪˈjuːrɪdeɪ, -diː/), a family that includes small or medium-size rodents. 
    The squirrel family includes tree squirrels, ground squirrels (including chipmunks and prairie dogs, among others), 
    and flying squirrels. Squirrels are indigenous to the Americas, Eurasia, and Africa, and were introduced by humans to Australia.[1] 
    The earliest known fossilized squirrels date from the Eocene epoch, and among other living rodent families, the squirrels are 
    most closely related to the mountain beaver and to the dormice.
    """
)

## Paměť
LLM modely jako takové si při odpovídání na uživatelův vstup předchozí interakce nepamatují. Proto se do nich musí explicitně přidat paměť, která bude historii v té či oné formě obsahovat.  
#### ConversationBufferMemory
Základním paměťovým objektem je *ConversationBufferMemory*. Ten si zkrátka pamatuje celou konverzaci tak, jak probíhala. Jak ale vypadá praktické použití? Asi nejsnazší je použit [*ConversationChain*](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/conversation/base.py). Do jeho konstruktoru vložíme chatovací model, paměť a lze sem umístit i flag, že požadujeme verbose výstup.  
BACHA - *ConversationBufferMemory* dovoluje v parametru *memory_key* měnit defaultní klíč (s hodnotou *history*), skrze který se dá dostat k paměti. Jenomže *ConversationalChain* s ničím takovým nepočítá a pokud nedostane *history*, tak způsobí pád.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

chat = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

conv_buffer_memory = ConversationBufferMemory(return_messages=False)

conversation = ConversationChain(
    llm=chat,
    verbose=True,
    memory=conv_buffer_memory
)

In [ ]:
langchain.debug = False

In [ ]:
conversation.invoke(input="Hi, who are you?")

In [ ]:
conversation.invoke(input="Do you know any famous hamsters?")

In [ ]:
conversation.invoke(input="Have you ever heard about hamster called 'Boo'?")

Na co slouží v definici paměťového objektu parametr *return_messages*? S pomocí metody *load_memory_variables* lze z paměťového objetku získat dosavadní historii konverzace (zde prázdné složené závorky coby parametr metody mají své použití pro určité typy pamětí, které nějaký vstupní parametr požadují). Pakliže bylo *return_messages* rovno False, obdržíme výstup v podobě velkého stringu:

In [ ]:
conv_buffer_memory.load_memory_variables({})

Pokud ale bude *return_messages* rovno True, má výstup metody složitější strukturu:

In [ ]:
conv_buffer_memory.load_memory_variables({})

V případě, že potřebujeme nastavit systémový prompt, bohužel musíme sáhnout po něčem komplikovanějším než je *ConversationChain* (byť v dalších příkladech pamětí budeme pro jednoduchost používat právě ten). Kód níže uvedený byl převzat (s mírnou úpravou) ze spodku [této stránky](https://python.langchain.com/docs/modules/memory/) dokumentace.  
Pozn.: pokud bude v tomto případě u paměti nastaveno *return_messages*=False, kód spadne. Pád je dán faktem, že *MessagesPlaceholder* potřebuje historii jako list konverzací, nikoli jako jeden string. Též je třeba, aby *variable_name* v *MessagePlaceholder* odpovídalo *memory_key* v paměti.

In [ ]:
from langchain.prompts import MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are an infantile AI assistant who start every message with 'peekaboo'."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
conv_buffer_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
    memory=conv_buffer_memory
)

In [ ]:
conversation({"question": "Hi, who are you?"})

In [ ]:
conversation({"question": "Do you know any famous hamsters?"})

#### ConversationBufferWindowMemory
*ConversationBufferWindowMemory* je variací na výše uvedený *ConversationBufferMemory*. Narozdíl od něj neobsahuje celou konverzaci, ale jen posledních *k* výměn (udáváme stejně se jmenujícím parametrem v konstruktoru paměti). Díky tomu je provoz chatbota levnější.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory 

chat = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

window_memory = ConversationBufferWindowMemory(k=1)

conversation = ConversationChain(
    llm=chat,
    verbose=False,
    memory=window_memory
)

In [ ]:
conversation.invoke(input="Hi, who are you?")

In [ ]:
window_memory.load_memory_variables({})

In [ ]:
conversation.invoke(input="Do you know any famous rabbits?")

In [ ]:
window_memory.load_memory_variables({})

#### ConversationTokenBufferMemory
Na podobném principu je založena i *ConversationTokenBufferMemory*, která uchovává historii jen dokud se vejde do *max_token_limit* počtu tokenů. Pokud nějaký příspěvek tuto hranici přesahuje, historie se de facto vymeže, tj. není to tak, že by v ní zbyl kousek starého příspěvku. Jelikož tokenizace je pro různé modely různá, musí se paměti v parametru *llm* předat i použitý model. Navíc je potřeba (minimálně pro OpenAI modely, ale možná i pro open source modely) mít nainstalovaný balíček [tiktoken](https://pypi.org/project/tiktoken/).

In [ ]:
from langchain.memory import ConversationTokenBufferMemory 

chat = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

token_memory = ConversationTokenBufferMemory(llm=chat, max_token_limit=60)

conversation = ConversationChain(
    llm=chat,
    verbose=False,
    memory=token_memory
)

In [ ]:
conversation.invoke(input="Hi, who are you?")

In [ ]:
token_memory.load_memory_variables({})

In [ ]:
conversation.invoke(input="Do you know any famous beavers?")

In [ ]:
token_memory.load_memory_variables({})

#### ConversationSummaryMemory
Odlišným typem paměti je *ConversationSummaryMemory*. U ní se provolává jazykový model (specifikovaný v konstruktoru paměti pod parametrem *llm*, tj. může mít jiné vlastnosti než "hlavní" v aplikaci používaný model), který má za úkol provést sumarizaci dosavadní historie konverzace a nové konverzační výměny mezi člověkem a strojem. Užitečné je to zejména u dlouhých kovnerzací, kde úspora z redukce délky historie přebije nutnost většího počtu provolávání modelu.

In [ ]:
from langchain_openai import OpenAI
from langchain.memory import ConversationSummaryMemory 

chat = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

summary_memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))

conversation = ConversationChain(
    llm=chat,
    verbose=False,
    memory=summary_memory
)

In [ ]:
conversation.invoke(input="Hi, who are you?")

In [ ]:
summary_memory.load_memory_variables({})

In [ ]:
conversation.invoke(input="Do you know what are differences between rhinos and unicorns?")

In [ ]:
summary_memory.load_memory_variables({})

In [ ]:
conversation.invoke(input="And what about similarities?")

In [ ]:
summary_memory.load_memory_variables({})

#### ConversationEntityMemory
Dalším speciálním typem paměti je *ConversationEntityMemory*. Ta z konverzace s pomocí jazykového modelu extrahuje informace o entitách.  
Abychom se vyhnuli chybové hlášce
```
Got unexpected prompt input variables. The prompt expects ['history', 'input'], but got ['entities', 'history'] as inputs from memory, and input as the normal input key. (type=value_error)
```
musíme do *ConversationChain* přidat nový parametr *prompt*, do kterého vložíme importováním získanou *ENTITY_MEMORY_CONVERSATION_TEMPLATE*

In [ ]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [ ]:
ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [ ]:
from langchain.memory import ConversationEntityMemory 
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

chat = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

entity_memory = ConversationEntityMemory(llm=OpenAI(temperature=0))

conversation = ConversationChain(
    llm=chat,
    verbose=False,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=entity_memory
)

In [ ]:
conversation.invoke(input="Hi, who are you?")

Pro zkontrolování obsahu paměti tentokrát nemůžeme použít *entity_memory.load_memory_variables({})*, ale musíme aplikovat *conversation.memory.entity_store.store*. Prozatím tam nic není.

In [ ]:
conversation.memory.entity_store.store

In [ ]:
conversation.invoke(input="Do you know what are differences between rhinos and unicorns?")

Postupně se ale začne pamět plnit dvojicemi "entita": "informace o entitě" (tady se to z nějakého důvodu nepovedlo...).

In [ ]:
conversation.memory.entity_store.store

In [ ]:
conversation.invoke(input="I read in a scientific journal that Duocorns, an unique breed of unicorns, have tow heads.")

In [ ]:
conversation.memory.entity_store.store

## Q&A nad dokumenty

#### Načtení dokumentů
##### Načtení pdfka

S klasickým chatbotem si sice můžeme povídat, ale chudák zná přinejlepším jen ty informace, které měl k dispozici při trénování. Co když s ním ale chceme diskutovat obsah našich pro něj v době učení neznámých dokumentů? Této problematice se budeme věnovat právě nyní.  
Pro začátek se bude naše množina dokumentů skládat z jednoho jediného kusu - z podmínek ke kreditním kartám od jedné z bank (nalezitelné [zde](https://www.kb.cz/cs/o-bance/dokumenty#13-Platebni-karty-debetni-a-kreditni)). Jedná se o pdf soubor. Pdfka ale umí langchain načítat až po nainstalování balíčku pypdf (přesněji různé langchainové metody mají různé prerekvizity, ale tahle cesta je asi nejméně komplikovaná).  
Nejprve vytvoříme instanci třídy PyPDFLoader a to sice tak, že do konstruktoru vložíme cestu k souboru. Bacha - musí se jednat o string, nikoli o Path! Na instanci posléze provoláme metodu *load*. Existuje sice i *load_and_split*, ale tu zde z pedagogických důvodů probírat nebudeme - leze totiž do zelí tématu probíranému o kus dál.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
file_path = "source_files\\podminky_debetnich_karet.pdf"

loader = PyPDFLoader(file_path)

In [ ]:
pages = loader.load()

Load nahraje krom obsahu stránek dokumentu i metadata. V nich máme název souboru a číslo stránky. Tj. vypadá to nějak takto:
```
[Document(page_content='  \n PODMÍNKY D EBETNÍCH KARET  ... výnosů z  trestné činnost i a financování te rorismu , ve znění \npozdějších předpi sů ', metadata={'source': 'source_files\\podminky_debetnich_karet.pdf', 'page': 0}),
 Document(page_content='PODMÍNKY DEBETNÍCH KARET ... poskytnutí dané Bankovní \nslužby.  \n ', metadata={'source': 'source_files\\podminky_debetnich_karet.pdf', 'page': 1}),
...
```
Pozn.: To, že u každé stránky - objektu typu *Document* - vidíme na začátku vždy "PODMÍNKY DEBENTNÍCH KARET", je dáno skutečností, že se zápatí stránky, které je vždy stejné, nějak dostalo na začátek. To mimo jiné znamená, že je potřeba dokumenty přes seriózním použitím začistit. Bohužel se tu též objevuje nadkritické množství mezer v místech, kde mezery neměly být (například hned v "PODMÍNKY D EBETNÍCH KARET").

Jiný způsob, jak načíst obsah pdf souborů, je založen na práci balíčku [unstructured](https://pypi.org/project/unstructured/). Při přípravě tohoto textu jsem (pokud si dobře pamatuji) pro pfd žádnou rozšířenou verzi (ala pip install "unstructured[pdf]") instalovat nemusel. Tudíž jsem nenarazil na potřebu mít Rust kompilátor kvůli jedné z prerekvizit - balíčku safetensors. Nicméně dodatečně bylo třeba nainstalovat balíčky pdf2image a pdfminer.six (nikoli pdfminer - ten už je neudržovaný, navíc snaha o jeho použití skončila s chybovou hláškou "ModuleNotFoundError: No module named 'pdfminer.high_level'").

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader

Unstructured loadery mohou fungovat ve dvou modech. Pokud bude parametr *mode* položet rovný "single", tak se celý dokument - zde pdfko - po použití metody *load* vrátí jako jeden langchainový *Document* objekt (tj. neproběhne ani rozdělení na stránky). Nicméně pokud se bude *mode* rovnat "elements", dojde k roztrhání dokumentu na malé kousky a ty kousky ponesou popisek charakterizují jejich obsah.

In [ ]:
loader = UnstructuredPDFLoader("source_files\\podminky_debetnich_karet.pdf", mode="elements")

In [ ]:
data = loader.load()

Zde máme pár příkladů:
```
[Document(page_content='PODMÍNKY DEBETNÍCH KARET', metadata={'source': 'source_files\\podminky_debetnich_karet.pdf', 'coordinates': {'points': ((239.57, 68.46839999999997), (239.57, 82.50839999999994), (456.64312, 82.50839999999994), (456.64312, 68.46839999999997)), 'system': 'PixelSpace', 'layout_width': 595.32, 'layout_height': 841.92}, 'filename': 'podminky_debetnich_karet.pdf', 'file_directory': 'source_files', 'last_modified': '2023-08-12T15:21:05', 'filetype': 'application/pdf', 'page_number': 1, 'category': 'Title'}),
 Document(page_content='Tyto Podmínky debetních karet obsahují bližší úpravu práv a povinností vyplývajících z uzavřené smlouvy, na základě které je poskytnuta debetní karta v souladu s pravidly příslušné Karetní společnosti. Seznamte se prosím důkladně s tímto dokumentem. Vaše případné dotazy rádi zodpovíme.', metadata={'source': 'source_files\\podminky_debetnich_karet.pdf', 'coordinates': {'points': ((56.64, 126.82999999999993), (56.64, 156.4699999999999), (541.018, 156.4699999999999), (541.018, 126.82999999999993)), 'system': 'PixelSpace', 'layout_width': 595.32, 'layout_height': 841.92}, 'filename': 'podminky_debetnich_karet.pdf', 'file_directory': 'source_files', 'last_modified': '2023-08-12T15:21:05', 'filetype': 'application/pdf', 'page_number': 1, 'category': 'NarrativeText'}),
  Document(page_content='Článek 1. Poskytnutí debetní karty a její obnova', metadata={'source': 'source_files\\podminky_debetnich_karet.pdf', 'coordinates': {'points': ((65.184, 169.61839999999995), (65.184, 180.65839999999992), (317.03912, 180.65839999999992), (317.03912, 169.61839999999995)), 'system': 'PixelSpace', 'layout_width': 595.32, 'layout_height': 841.92}, 'filename': 'podminky_debetnich_karet.pdf', 'file_directory': 'source_files', 'last_modified': '2023-08-12T15:21:05', 'filetype': 'application/pdf', 'page_number': 1, 'category': 'NarrativeText'}),
 Document(page_content='1.1', metadata={'source': 'source_files\\podminky_debetnich_karet.pdf', 'coordinates': {'points': ((72.504, 195.3499999999999), (72.504, 204.3499999999999), (87.48599999999999, 204.3499999999999), (87.48599999999999, 195.3499999999999)), 'system': 'PixelSpace', 'layout_width': 595.32, 'layout_height': 841.92}, 'filename': 'podminky_debetnich_karet.pdf', 'file_directory': 'source_files', 'last_modified': '2023-08-12T15:21:05', 'filetype': 'application/pdf', 'page_number': 1, 'category': 'UncategorizedText'}),
...
 Document(page_content='VER DDT_PODMPKEV.PDF', metadata={'source': 'source_files\\podminky_debetnich_karet.pdf', 'coordinates': {'points': ((487.18, 807.3276), (487.18, 811.2876), (539.8808799999999, 811.2876), (539.8808799999999, 807.3276)), 'system': 'PixelSpace', 'layout_width': 595.32, 'layout_height': 841.92}, 'filename': 'podminky_debetnich_karet.pdf', 'file_directory': 'source_files', 'last_modified': '2023-08-12T15:21:05', 'filetype': 'application/pdf', 'page_number': 10, 'category': 'Title'})]
```
Všimněme si, že kousky textu jsou seřazeny podle toho, jak se v pdfku objevily. V metadatech nalezneme mimo jiné číslo stránky, čtyři body ohraničující lokaci textu, jméno souboru anebo kategorii textu.  
Jaké kategorie tu vidíme? 
- Title - nevyskytuje se jen u nadpisů, ale například i u kapitálkami vyvedeného zápatí ("VER DDT_PODMPKEV.PDF" z konce příkladu)
- NarrativeText - většinově se jedná o občejný text
- UncategorizedText - zde se obvykle nalézá "smetí" - číslování stránek, čísla paragrafů stojící mimo text anebo zápatí.

Rozdělení na kategorie nicméně není úplně spolehlivé. Když bychom si u našeho dokumentu nechali vypsat všechny "Title" dokumenty kódem
```python
for one_doc in data:
    if one_doc.metadata["category"] == "Title":
        print(one_doc.page_content)
```
najdeme tam mezi nadpisy a kousky zápatí i normální text, např. "3D Secure. Všechny námi poskytnuté debetní karty jsou 3D Secure aktivní.". A naopak v "NarrativeText" člověk nalezne pár nadpisů. Je nakonec otázkou, zda není lepší, když si člověk dokument prohlédne a začistí ho ručně podle svého. Nakonec tak bude nejvhodnější single mode unstructured (narozdíl od PyPDFLoaderu nemá nabytečné mezery)

In [ ]:
loader = UnstructuredPDFLoader("source_files\\podminky_debetnich_karet.pdf", mode="single")
data = loader.load()

Vytvoření objektu typu Document potřebné pro ruční vytváření fragmentů:

In [ ]:
from langchain_core.documents.base import Document

fragment = Document(
    page_content="this is fragment text",
    metadata={"file":"something.pdf", "page":0, "another_metadata_field":"something"}
)

##### Načtení html souboru
Pro načítání html stránek existují v rámci Langchainu dvě podporované cesty. Jedna vyžaduje použití balíčku [unstructured](https://pypi.org/project/unstructured/). Jako data použijeme wiki stránku o [jednom druhu křečka](https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster). Zdůrazněme, že následující postup se týka html souboru uloženého na disku, nikoli webové stránky - na to je jiný loader.

In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

In [ ]:
loader = UnstructuredHTMLLoader("source_files\\Winter white dwarf hamster - Wikipedia.htm")

In [ ]:
data = loader.load()

Nahraný objekt vypadá (po vyřazení většiny textu z důvodu přehlednosti) takto:
```
[Document(page_content='Toggle the table of contents\n\nToggle the table of contents\n\nWinter white dwarf hamster\n\n43 languages\n\nالعربية\n\nAsturianu\n\nБългарски\n\nBrezhoneg\n\nCatalà\n\nCebuano\n\nČeština\n\nDeutsch\n\nDiné bizaad\n\nEesti\n\nEspañol\n\nEuskara\n\nفارسی\n\nFrançais\n\nFrysk\n\n한국어\n\nՀայերեն\n\nHrvatski\n\nBah  
...  
nCategories:\n\nIUCN Red List least concern species\n\nPhodopus\n\nRodents of Asia\n\nMammals described in 1773\n\nMammals of Siberia\n\nTaxa named by Peter Simon Pallas\n\nHidden categories: \n\nArticles with short description\n\nShort description is different from Wikidata\n\nGood articles\n\nArticles with \'species\' microformats', metadata={'source': 'source_files\\Winter white dwarf hamster - Wikipedia.htm'})]
```
Před použitím bude tedy potřeba provést opravdu masivní začištění.

Opět je možné použít unstructured načítání v modu "elements".

In [ ]:
loader = UnstructuredHTMLLoader("source_files\\Winter white dwarf hamster - Wikipedia.htm", mode="elements")

In [ ]:
data = loader.load()

Výsledek vypadá takto:
```
[Document(page_content='Toggle the table of contents', metadata={'source': 'source_files\\Winter white dwarf hamster - Wikipedia.htm', 'filename': 'Winter white dwarf hamster - Wikipedia.htm', 'file_directory': 'source_files', 'last_modified': '2023-08-22T20:35:12', 'filetype': 'text/html', 'page_number': 1, 'emphasized_text_contents': ['Toggle the table of contents'], 'emphasized_text_tags': ['span'], 'category': 'Title'}),  
...  
 Document(page_content='population density is highly varied.', metadata={'source': 'source_files\\Winter white dwarf hamster - Wikipedia.htm', 'filename': 'Winter white dwarf hamster - Wikipedia.htm', 'file_directory': 'source_files', 'last_modified': '2023-08-22T20:35:12', 'filetype': 'text/html', 'page_number': 3, 'link_urls': ['https://en.wikipedia.org/wiki/Population_density'], 'link_texts': ['population density'], 'category': 'NarrativeText'}),
 Document(page_content='[23]', metadata={'source': 'source_files\\Winter white dwarf hamster - Wikipedia.htm', 'filename': 'Winter white dwarf hamster - Wikipedia.htm', 'file_directory': 'source_files', 'last_modified': '2023-08-22T20:35:12', 'filetype': 'text/html', 'page_number': 3, 'link_urls': ['#cite_note-J1979:R1998:E-23'], 'link_texts': ['[23]'], 'category': 'UncategorizedText'}),  
 ...  
Document(page_content="Articles with 'species' microformats", metadata={'source': 'source_files\\Winter white dwarf hamster - Wikipedia.htm', 'filename': 'Winter white dwarf hamster - Wikipedia.htm', 'file_directory': 'source_files', 'last_modified': '2023-08-22T20:35:12', 'filetype': 'text/html', 'page_number': 3, 'link_urls': ['https://en.wikipedia.org/wiki/Category:Articles_with_%27species%27_microformats'], 'link_texts': ["Articles with 'species' microformats"], 'category': 'ListItem'})]
```

Pro načtení html souboru lze též použít loader využívající BeautifulSoup4. Moje první provolání metody *load* vedlo k chybové hlášce obsahující mimo jiné
```
soup = BeautifulSoup(f, **self.bs_kwargs)
```
a
```
FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
```
A to i když byl ve virtuálním prostředí balíček BeautifulSoup4 nainstalován. Co ale chybělo byl balíček [lxml](https://pypi.org/project/lxml/). Po jeho nainstalování se ale (u anglického!) textu objevila další chybová hláška:
```
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 27841: character maps to <undefined>
```
To kvůli tomu, že jsem nepoužil parametr *open_encoding* a html soubor se tak otevíral s defaultním kódováním, kterým je utf-8.

In [ ]:
from langchain_community.document_loaders import BSHTMLLoader
loader = BSHTMLLoader("source_files\\Winter white dwarf hamster - Wikipedia.htm", open_encoding="latin1")
data = loader.load()

Opět z nahraného objektu ukážeme pouze začátek a konec:
```
[Document(page_content='\n\n\nWinter white dwarf hamster - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload
...
\nCookie statement\n\n\n\n\n\n\n\n\n\n\n\n\nToggle limited content width\n\n\n\n\n\n\n', metadata={'source': 'source_files\\Winter white dwarf hamster - Wikipedia.htm', 'title': 'Winter white dwarf hamster - Wikipedia'})]
```
Zdá se, že potřeba začištění je tu ještě větší než u balíčku unstructured...

##### [Načtení webové stránky](https://python.langchain.com/docs/integrations/document_loaders/web_base)

V praxi bychom upřednostňovali, kdybychom vytěžované stránky nemuseli stahovat a kdyby si Langchain informace natahal přímo z webu. Na to slouží *WebBaseLoader*.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster")

In [ ]:
data = loader.load()

Zde vidíme příklad formátu, v jakém je stránka načtena:
```
[Document(page_content='\n\n\n\nWinter white dwarf hamster - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\nLanguages\n\nLanguage links are at the top of the page across from the title.
...
\nContact Wikipedia\nCode of Conduct\nMobile view\nDevelopers\nStatistics\nCookie statement\n\n\n\n\n\n\n\n\n\n\n\n\nToggle limited content width\n\n\n\n\n\n\n\n', metadata={'source': 'https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster', 'title': 'Winter white dwarf hamster - Wikipedia', 'language': 'en'})]
```

V rámci jednoho volání lze načíst i více stránek:

In [ ]:
loader = WebBaseLoader([
    "https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster", 
    "https://en.wikipedia.org/wiki/Syrian_hamster"
])

In [ ]:
data = loader.load()

Ukázka výsledku:
```
[Document(page_content='\n\n\n\nWinter white dwarf hamster - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute
...
\nContact Wikipedia\nCode of Conduct\nMobile view\nDevelopers\nStatistics\nCookie statement\n\n\n\n\n\n\n\n\n\n\n\n\nToggle limited content width\n\n\n\n\n\n\n\n', metadata={'source': 'https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster', 'title': 'Winter white dwarf hamster - Wikipedia', 'language': 'en'}),
 Document(page_content='\n\n\n\nGolden hamster - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file
...
\nCode of Conduct\nMobile view\nDevelopers\nStatistics\nCookie statement\n\n\n\n\n\n\n\n\n\n\n\n\nToggle limited content width\n\n\n\n\n\n\n\n', metadata={'source': 'https://en.wikipedia.org/wiki/Syrian_hamster', 'title': 'Golden hamster - Wikipedia', 'language': 'en'})]
```

I pro načítání stránek z webu existuje [unstructured loader](https://python.langchain.com/docs/integrations/document_loaders/url).

In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [ ]:
urls = [
    "https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster", 
    "https://en.wikipedia.org/wiki/Syrian_hamster"
]

Když člověk nemá nainstalovaný balíček libmagic, objeví se hlášky
```
libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
Error fetching or processing https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster, exception: Invalid file. The FileType.UNK file type is not supported in partition.
libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
Error fetching or processing https://en.wikipedia.org/wiki/Syrian_hamster, exception: Invalid file. The FileType.UNK file type is not supported in partition.
```
Načtený objekt pak bude prázdným listem.
Problém je, že existuje více libmagic ([zde](https://pypi.org/project/python-libmagic/#description) a [zde](https://pypi.org/project/libmagic/#description)), ty jsou ale relativně staré a prakticky od svého vzniku neudržované...  
Též existuje balíček [python-magic](https://pypi.org/project/python-magic/#history). Ten už vypadá živěji, nicméně když nainstalujeme jeho nejnovější verzi, obdržíme errory
```
Error fetching or processing https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster, exception: module 'magic' has no attribute 'from_buffer'
Error fetching or processing https://en.wikipedia.org/wiki/Syrian_hamster, exception: module 'magic' has no attribute 'from_buffer'
```
Instalace starší verze vede též k chybovým hláškám.  
Problém lze vyřešit instalací [python-magic-bin](https://pypi.org/project/python-magic-bin). Nicméně jedná se o binárky z repa zapadaného prachem.

In [ ]:
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

Ukázka:
```
[Document(page_content='Toggle the table of contents\n\nToggle the table of contents\n\nWinter white dwarf hamster\n\n43 languages\n\nالعربية\n\nAsturianu\n\nБългарски\n\nBrezhoneg\n\nCatalà\n\nCebuano\n\nČeština\n\nDeutsch\n\nDiné bizaad\n\nEesti\n\nEspañol\n\nEuskara\n\nفارسی\n\nFrançais\n\nFrysk\n\n한국어\n\nՀայերեն\n\nHrvatski\n\nBahasa Indonesia\n\nItaliano\n\nעברית\n\nҚазақша\n\nKotava\n\nLatviešu\n\nMagyar\n\nمصرى\n\nNederlands\n\n日本語\n\nNorsk bokmål\n\nNorsk nynorsk\n\nPolski\n\nPortuguês\n\nRomână\n  
...  
Red List least concern species\n\nPhodopus\n\nRodents of Asia\n\nMammals described in 1773\n\nMammals of Siberia\n\nTaxa named by Peter Simon Pallas\n\nHidden categories: \n\nArticles with short description\n\nShort description is different from Wikidata\n\nGood articles\n\nArticles with \'species\' microformats', metadata={'source': 'https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster'}),
 Document(page_content='Toggle the table of contents\n\nToggle the table of contents\n\nGolden hamster\n\n49 languages\n\nالعربية\n\nAragonés\n\nБеларуская (тарашкевіца)\n\nБългарски\n\nCatalà\n\nCebuano\n\nČeština\n\nCymraeg\n\nDeutsch\n\nDiné bizaad\n\nEesti\n\nEspañol\n\nEsperanto\n\nEuskara\n\nفارسی\n\nFrançais\n\nFrysk\n\nGaeilge\n\n한국어\n\nHrvatski\n\nBahasa Indonesia\n\nIta  
 ...  
 abic-language text\n\nAll articles with unsourced statements\n\nArticles with unsourced statements from June 2019\n\nArticles with unsourced statements from September 2019\n\nCommons link is on Wikidata\n\nArticles with GND identifiers\n\nArticles with J9U identifiers\n\nArticles with LCCN identifiers\n\nArticles with NKC identifiers\n\nArticles containing video clips', metadata={'source': 'https://en.wikipedia.org/wiki/Syrian_hamster'})]
```

In [ ]:
loader = UnstructuredURLLoader(urls=urls, mode="elements")

In [ ]:
data = loader.load()

Ukázka:
```
[Document(page_content='Toggle the table of contents', metadata={'filetype': 'text/html', 'page_number': 1, 'url': 'https://en.wikipedia.org/wiki/Winter_white_dwarf_hamster', 'emphasized_text_contents': ['Toggle the table of contents'], 'emphasized_text_tags': ['span'], 'category': 'Title'}),  
...  
 Document(page_content='A hamster wheel is a common type of environmental enrichment, and it is important that hamsters have a wheel in their cage. TVT recommends wheels should be at least 30\xa0cm for Syrian hamsters, since smaller diameters lead to permanent spinal curvatures, especially in young animals. They also recommend a solid running surface because rungs or mesh can cause injury.[19] A hamster should be able to run on its wheel without arching its back. A hamster that has to run with an arched back can have back pain and spine problems. A variety of toys and cardboard tubes and boxes can help to provide enrichment, as they are energetic and need space to exercise.[20]', metadata={'filetype': 'text/html', 'page_number': 3, 'url': 'https://en.wikipedia.org/wiki/Syrian_hamster', 'link_urls': ['/wiki/Hamster_wheel', '#cite_note-19', '#cite_note-20'], 'link_texts': ['hamster wheel', '[19]', '[20]'], 'category': 'NarrativeText'}),
 Document(page_content='Most hamsters in American and British pet stores are golden hamsters. Originally, golden hamsters occurred in just one color – the mixture of brown, black, and gold, but they have since developed a variety of color and pattern mutations, including cream, white, blonde, cinnamon, tortoiseshell, black, three different shades of gray, dominant spot, banded, and dilute.[citation needed]', metadata={'filetype': 'text/html', 'page_number': 3, 'url': 'https://en.wikipedia.org/wiki/Syrian_hamster', 'link_urls': ['/wiki/Wikipedia:Citation_needed'], 'link_texts': [None], 'emphasized_text_contents': ['citation needed', 'citation needed'], 'emphasized_text_tags': ['i', 'span'], 'category': 'NarrativeText'}),  
...  
Document(page_content='Articles containing video clips', metadata={'filetype': 'text/html', 'page_number': 3, 'url': 'https://en.wikipedia.org/wiki/Syrian_hamster', 'link_urls': ['/wiki/Category:Articles_containing_video_clips'], 'link_texts': ['Articles containing video clips'], 'category': 'ListItem'})]
```

#### Splittery
Obsah stránek by zejména pro výživnější texty mohl být větší, než počet tokenů, které dokáže v jeden okamžik jazykový model zpracovat. Proto se dokumenty musí rozdělit na malé kousky.  
Základním nástrojem pro takovou úlohu je *CharacterTextSplitter*. V jeho konstruktoru musíme do parametru *separator* vložit separační znak (resp. posloupnost znaků). Bacha - *CharacterTextSplitter* podporuje jen a pouze jeden separátor. Pokud bude splitování probíhat podle nějakého speciálního escapovaného znaku (např. podle znaku nového řádku) a nepoužíváme regulární výrazy (tj. parametr *is_separator_regex* není položený rovný True), musíme psát jen jedno zpětné lomítko! Následně specifikujeme optimální velikost fragmentu textu (parametr *chunk_size*; číslo je v počtu znaků podle funkce *len*) a velikost překryvu mezi fragmenty (parametr *chunk_overlap*).

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
chunk_size = 250
chunk_overlap = 50

In [ ]:
char_text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
)

Pokud bychom chtěli splittovat obyčejný text, použili bychom metodu *split_text* instance *CharacterTextSplitter*, přičemž inkriminovaný text by byl předán metodě jako parametr. Nicméně my máme list obsahující navíc krom samotného textu i metadata o stránkách původního pdfka, s čímž není *split_text* očekávající string kompatibilní. Proto musíme použít metodu *split_documents*.  
Jak vlastně splitování funguje?
 - pokud separátor není v dokumentu (v našem případě dokument = jedna stránka) přítomen, k žádnému rozsekání nedochází. Tj. pokud na vstupu bylo 10 stránek, bude stejný počet i na výstupu, což pro praktické použití optimální není.  
 - pokud je separátor vzácný, tak se některé dokumenty vůbec nerozsekají (protože v nich separátor není), jiné se rozdělí třeba jen podle jediného výskytu separátoru (i když i pak budou oba vzniklé fragmenty větší než *chunk_size*)
 - pokud je separátor přítomný často, proběhne rozsekávání tak, aby byla téměř dosažena, ale nikdy překročena *chunk_size* (resp. možná se dokument rozseká podle všech výskytů separátoru, ale pak se na sebe některé fragmenty znova nalepí).

Bacha, pokud používáme regexy (*is_separator_regex* = True) a zvolíme nějaký obecný pattern (třeba "\\w"), tak se nám tímto patternem nahradí všechny relevantní znaky nahradí. Tj. fragment pak třeba vypadá takto:
```
'\\w \\w \\w \\w   \n \n \n \n \n\\w \\w \\w, \\w. \\w., \\w \\w:  \n\\w \\w, \\w \\w \\w \\w \\w. \\w, \\w \\w \\w, \\w: \\w  \n\\w \\w \\w \\w \\w \\w \\w \\w \\w \\w, \\w \\w, \\w \\w \\w \\w/\\w  \n\\w \\w \\w.\\w \n \\w \\w \\w \\w \\w \\w \\w \\w \\w \\w \\w \\w  \\w \\w, \\w \\w \n\\w \\w \\w  \\w \\w \\w  \\w \\w'
```
Takovému chování zabráníme umístěním parametru *keep_separator*=True do konstruktoru *CharacterTextSplitter*u.  
Níže vidíme příklad výstupu metody *split_documents*:

In [ ]:
char_splits = char_text_splitter.split_documents(pages)
print(char_splits[0])
print(len(char_splits[0].page_content))

Výše jsme viděli, že *CharacterTextSplitter* není pro práci s obyčejným textem kvůli omezení na jeden separátor moc praktický. Proto je vhodnější použít *RecursiveCharacterTextSplitter*. Ten nemá parametr *separator*, nýbrž *separators*. Do něj se vkládá list potenciálních separátorů. Jeho defaultní hodnotou je ["\n\n", "\n", " ", ""]. Myšlenka za touto volbou je taková, že se spitter snaží držet pohromadě odstavce (a potom věty a pak slova) tak dlouho, jak to jen jde, kvůli zachování informace schované v textu.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
chunk_size = 250
chunk_overlap = 50

In [ ]:
rec_text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
)

In [ ]:
rec_splits = rec_text_splitter.split_documents(pages)

In [ ]:
rec_splits[0]

V případě, že bychom chtěli pracovat nikoli s přirozeným jazykem, ale s počítačovým kódem, mohou nám pomoci již předpřipravené separátory pro několik hlavních programovacích jazyků. Ty získáme naimportováním *Language* z *langchain.text_splitter*. Pokud separátory chceme vidět, musíme použít *get_separators_for_language*: 

In [ ]:
from langchain.text_splitter import Language

In [ ]:
print(RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON))

Při samotném praktickém použití vložíme *Language.PYTHON* do parametru *language* metody *from_language* (viz příklad převzatý z dokumentace):

In [ ]:
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

#### Embeddings, vectorstore
Při hledání odpovědi na určitou otázku de facto hledáme otázce nejpodobnější fragmenty dokumentů. Nicméně jak bychom v přirozeném jazyce vůbec podobnost definovali? Nejpraktičtější řešení spočívá v převodu splitováním vyrobených fragmentů dokumentů na vektory (posloupnosti čísel). S nimi už můžeme pro (ne)podobnost aplikovat klasický matematický aparát ala cosinová podobnost. Tyto vektory se ale též musí někam uložit. Proto mluvíme obecně o tzv. vector stores, přičemž zde budeme konkrétně používat Chromu.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
file_path = "source_files\\podminky_debetnich_karet.pdf"

loader = PyPDFLoader(file_path)
pages = loader.load()

chunk_size = 250
chunk_overlap = 50

rec_text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
)

rec_splits = rec_text_splitter.split_documents(pages)

Napřed specifikujeme, jaký model budeme používat pro výpočet embeddingů. *OpenAIEmbeddings* znamená, že použijeme model text-embedding-ada-002 od OpenAI. Ten při klasickém používání ala chatování či doplňování textu není tak mocný jako GPT3.5 nebo aspoň DaVinci. Nicméně při nápočtu embeddingů to zas takovou roli nehraje a naopak se hodí, že je tento model řádově levnější.

In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
embedding = OpenAIEmbeddings()

Pozn.: v případě práce s Azure OpenAI použijeme *AzureOpenAIEmbeddings*. Musí se specifikovat deployment, ve kterém je nasazený model určený na embeddingování. Taktéž je třeba nastavit *chunk_size* na jedničku (jinak se objeví tuším "too many inputs" chybová hláška).
```python
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(deployment="deplyment_name_s_embedding_modelem", chunk_size=1)
```

Pro použití Chromy musíme napřed nainstalovat odpovídající balíček [chromadb](https://pypi.org/project/chromadb/) (pokud se instalace nezdaří, máte možná příliš novou verzi Pythonu - v době psaní těchto řádků byla verze 3.10 ok, ale 3.12 už ne). Následně je potřeba provést importování.

In [ ]:
from langchain.vectorstores import Chroma

Chroma je souborová databáze. Musíme tedy specifikovat, kam se budou její soubory (fakticky jde o parquety) ukládat.

In [ ]:
simple_doc_dir = 'embeddings/simple/'

In [ ]:
vectordb = Chroma.from_documents(
    documents=rec_splits,
    embedding=embedding,
    persist_directory=simple_doc_dir
)

Na následující příkaz nesmíme zapomenout, neboť bez něj by se vektory fakticky neuložily na disk.

In [ ]:
vectordb.persist()

Pro načtení uložených embeddingů se použije

In [ ]:
vectordb = Chroma(
    persist_directory=simple_doc_dir, 
    embedding_function=embedding
)

V běžném provozu asi nebudeme následující příkazy krom občasných kontrol potřebovat. Nicméně ne vždy se člověk v běžném provozu pohybuje...  
Pro počet záznamů v konkrétním vectorstoru použijeme metodu *\_collection.count*.

In [ ]:
print(vectordb._collection.count())

Metoda *peek* ukáže několik prvních záznamů (počet je specifikován parametrem *limit*, přičemž defaultní hodnota má velikost 10).
```
vectordb._collection.peek(limit=3)
```
Výstup metody není z nejmenších, proto ho zde v surové podobě neukazujeme. Jedná se o slovník, ve kterém jsou postupně tyto klíče a hodnoty:
- ids s listem idček jednotlivých fragmentů dokumentů  
- embeddings s listem vektorů (alias listů floatů), ve kteréch jsou fragmenty dokumentů zakódované  
- metadatas s listem metadat, která nabývají podoby slovníků s klíči "page" a "source"
- documents s listem samotných fragmentů dokumentů v textové podobě
```
{'ids': ['a737d8ec-39ea-11ee-8841-80ce622bc396',
  'a73b7183-39ea-11ee-a74a-80ce622bc396',
  'a73b7184-39ea-11ee-bd3c-80ce622bc396'],
 'embeddings': [[0.0016977092018350959,
   -0.013771715573966503,
   0.0062872255221009254,
   ....
   ....
   -0.0042107910849153996,
   -0.027450013905763626,
   -0.02284945175051689,
   0.06665701419115067,
   ...]],
 'metadatas': [{'page': 0,
   'source': 'source_files\\podminky_debetnich_karet.pdf'},
  {'page': 0, 'source': 'source_files\\podminky_debetnich_karet.pdf'},
  {'page': 0, 'source': 'source_files\\podminky_debetnich_karet.pdf'}],
 'documents': ['PODMÍNKY D EBETNÍCH KARET   \n \n \n \n \nKomerční bank a, a. s., se sídlem:  \nPraha 1, Na Pří kopě 33 čp. 969, PSČ 114 07, IČO: 45317054  \nZAPSAN Á V OBCHODNÍM REJSTŘÍKU VEDENÉM MĚSTSKÝM SOUDEM V PRAZE, ODDÍL B, VLOŽKA 1 360 1/10',
  'VER DDT_ PODMPKEV.PDF \n Tyto Podmínky debetních karet obsahují bližší úpravu práv a povinností vyplývajících z  uzavřené smlouvy, na základě',
  'které je poskytnuta  debetní karta v  souladu s  pravidly příslušné Karetní společnosti.  Seznamte se prosím důkladně \ns tímto dokumentem. Vaše p řípadné dotazy rádi zodpovíme.  \n \nČlánek 1. Poskytnutí  debetní karty a její obnova']}
```

Metoda get (výsledek je stejný pro *get* i pro *\_collection.get*) slouží k získání buďto některých záznamů (to tehdy, když do parametru ids vložíme string či list stringů s idčky) anebo všech záznamů (když idčka nespecifikujeme; počet vrácených fragmentů lze omezit parametrem *limit*) 

In [ ]:
vectordb.get(ids="a73b7184-39ea-11ee-bd3c-80ce622bc396")

Defaultně se v odpovědi neobjeví embeddingy. Toto chování je řízené parametrem *include*, který defaultně obsahuje ["metadatas", "documents"]. Tudíž pokud chceme embeddingy, musíme je do listu explicitně přidat.
```
vectordb.get(
    ids="a73b7184-39ea-11ee-bd3c-80ce622bc396",
    include=["metadatas", "documents", "embeddings"]
)
```

Pokud chceme realizovat filtrování na základě metadat, musíme použít parametr *where*. Do něj vložíme slovník, kde klíčem bude "page" nebo "source" a hodnotou číslo stránky či zdrojový soubor>
```
vectordb.get(where={"page": 0})
```
```
vectordb.get(where={"source": "source_files\\podminky_debetnich_karet.pdf"})
```

Pro získávání nejpodobnějších dokumentů vstupnímu textu sice slouží metoda *query* s parametrem *query_texts*, nicméně my v praxi budeme spíše používat langchainovou nadstavbu nad touto metodou.

Co musíme udělat, když chceme některý ze záznamů updatovat? Vezměme si náhodně fragment i idčkem "a73b7184-39ea-11ee-bd3c-80ce622bc396".

In [ ]:
some_fragment = vectordb.get(ids="a73b7184-39ea-11ee-bd3c-80ce622bc396")

Jeho metadata vypadají takto:

In [ ]:
some_fragment["metadatas"]

Nyní metadata přepíšeme - přidáme další pole.

In [ ]:
some_fragment["metadatas"] = [{
    'page': 0, 
    'source': 'source_files\\podminky_debetnich_karet.pdf',
    "some_info": "Terms of use of debit cards"
}]

Provedeme *update* stejnejmennou metodou (bacha, musí to být v *\_collection* - *vectordb.update* se vztahuje na objekty typu *Document*). Prvním parametrem je idčko, poté zde máme parametr *metadatas*, do kterého vložíme nová metadata. Mohl by tu být i parametr embeddings (pro update embeddingů) či documents (pro úpravu samotných textů).

In [ ]:
vectordb._collection.update("a73b7184-39ea-11ee-bd3c-80ce622bc396", metadatas=some_fragment["metadatas"])

Update se opravdu provedl.

In [ ]:
vectordb.get(ids="a73b7184-39ea-11ee-bd3c-80ce622bc396")

Přičemž zbylé záznamy zůstávají touto operací nepostižené.

In [ ]:
vectordb.get(ids="a73b7183-39ea-11ee-a74a-80ce622bc396")

Nakonec nesmíme zapomenout na uložení změn.

In [ ]:
vectordb.persist()

Podobným způsobem by probíhalo i mazání záznamů, kdy bychom použili metodu *delete*:
```
vectordb._collection.delete(ids="a73b7183-39ea-11ee-a74a-80ce622bc396")
```

#### Similarity search
Řekli jsme si, že metodu *query* na hledání podobnosti dokumentů používat nebudeme. Co ale místo ní máme k dispozici?

In [ ]:
question = "Jak dlouho debentní karta platí?"

Metoda *similarity_search* vyhledává *k* nejpodobnějších fragmentů dokumentů k dokumentu vloženému do parametru *query*. Založená je na počítání cosinové podobnosti mezi vektory fragmentů.

In [ ]:
simple_sim_docs = vectordb.similarity_search(query=question,k=3)

In [ ]:
simple_sim_docs

Lze do ní přidat i parametr *filter*, který se hodí pro omezení množiny, ve které se podobné framenty hledají, a to sice na základě metadat.

In [ ]:
simple_sim_docs = vectordb.similarity_search(
    query=question,
    k=3,
    filter={"page":0}
)

In [ ]:
simple_sim_docs

Metoda *similarity_search* u vrácených fragmentů neukazuje, jak moc jsou dokumentu v *query* podobné. Tuto informaci můžeme ale dostat s metodou *similarity_search_with_score* - ta vrací velikost cosinové podobnosti, tj. menší číslo je lepší a nejlepší je 0.

In [ ]:
sim_score_docs = vectordb.similarity_search_with_score(query=question,k=3)
sim_score_docs

Podobně funguje *similarity_search_with_relevance_scores*, pouze je cosinová podobnost přepočítána na relevance score. To se nalézá v uzavřeném intervalu mezi 0 a 1, přičemž 0 jsou zcela nepodobné dokumenty, zatímco 1 mají zcela identické dokumenty.

In [ ]:
sim_rel_score_docs = vectordb.similarity_search_with_relevance_scores(query=question,k=3)
sim_rel_score_docs

Na poněkud odlišném principu funguje *max_marginal_relevance_search*. Tato metoda se snaží současně optimalizovat podobnost a diverzitu dokumentů. To v praxi znamená, že najde na základě cosinové podobnosti *fetch_k* fragmentů (defaultně 20). Poté je ale seřadí tak, že penalizuje podobné či dokonce identické dokumenty a z nového řazení vrátí *k* fragmentů (defaultně 4). Bohužel zdá se, že žádné číslo kvantifikující podobnost či kvalitu výběru není ve funkci dostupné.

In [ ]:
max_relev_docs = vectordb.max_marginal_relevance_search(query=question,k=3, fetch_k=20)
max_relev_docs

Specialitou je použití komprese na výstupních fragmentech. Fragmenty mohou totiž obsahovat značné množství informací pro určitý dotaz nerelevantních. Užitečné informace se potom v záplavě zbytečností snadno ztratí. 

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

Abychom viděli, co Langchain dělá, zapneme debugování.

In [ ]:
langchain.debug = True

Samotnou kompresi (či přesněji osekávání zbytečných slov) realizuje fakticky jazykový model, musíme tudíž vytvoři jeho instanci.

In [ ]:
chat = ChatOpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(chat)

Následně vytvoříme samotný kompresor.

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

Na něm provoláme metodu *get_relevant_documents*. Bohužel jsem nepřišel na to, jak specifikovat počet vrácených fragmentů (parametr *k* nefunguje), takže se vrací defaultní 4.  
"Kompresi" zajišťuje následující prompt:
```
Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. \n\nRemember, *DO NOT* edit the extracted parts of the context.\n\n> Question: Jak dlouho debentní karta platí?\n> Context:\n>>>\nplatíte  předem a  účtujeme  jej nejdříve 5.  obchodní den  po sjednání karty. V dalších měsících  po dobu \nplatnosti karty  je cena splatná  ve stejný den . Pokud den splatnosti připa dne na den, který není Obchodním\n>>>\nExtracted relevant parts:
```

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents(query=question)

In [ ]:
compressed_docs

In [ ]:
langchain.debug = False

#### Question answering
Uživatel asi nebude chtít, aby mu stroj na jeho otázku vyplivl více či méně souvislé fragmenty dokumentů. Bude si spíše přát odpověď v přirozeném jazyce. Této úloze se budeme věnovat nyní.  
Nejprve si připravme věci, se kterými jsme se seznámili již v předchozík podkapitolách.

In [ ]:
simple_doc_dir = 'embeddings/simple/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=simple_doc_dir, embedding_function=embedding)
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

Následně si vytvoříme chain pro question answering - [RetrievalQA](https://python.langchain.com/docs/use_cases/question_answering/how_to/vector_db_qa).

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever()
)

Otázku do něj vložíme takto:

In [ ]:
question = "Jak dlouho debentní karta platí?"
result = qa_chain({"query": question})

A obdržíme následující slovník:

In [ ]:
result

Ok, odpověď v přirozeném jazyce jsme dostali. Co když bychom ale chtěli upravit systémový prompt? A jak si zobrazíme fragmenty, ze kterých jazykový model skládal odpověď?

In [ ]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Answer in Czech language. 
{context}
Question: {question}
Helpful Answer:"""
qa_chain_prompt = PromptTemplate.from_template(template)

Zdrojové fragmenty dostaneme díky parametru *return_source_documents* rovnému True. Šablonu se svého druhu systémovým promptem pak do chainu vložíme skrze parametr *chain_type_kwargs*. Ten obsahuje slovník, ve kterém musí být pro klíč "prompt" vložena námi specifikovaná šablona.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_chain_prompt}
)

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result

Určité parametry můžeme vložit i do *vectordb.as_retriever()*. Například pokud bychom chtěli přejít od defaultního similarity searche na max marginal relevance search, přidáme tam parametr *search_type* s hodnotou "mmr". Další paramtery už nejsou samostatné, nýbrž se nalézají ve slovníku v parametru *search_kwargs*. Může jít o počet vrácených fragmentů (parametr *k*), *fetch_k* pro MMR, *score_threshold* pro similarity search na vrácení dostatečně kvalitích fragmentů či v případě filtrování dle metadat parametr filter.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 20}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_chain_prompt}
)
result = qa_chain({"query": question})
result

Pokud chceme mít chatbota, se kterým lze diskutovat déle než jednu dialogovou výměnu, musíme si vytvořit paměťový objekt.

In [ ]:
from langchain.memory import ConversationBufferMemory
conv_buff_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

Je třeba též použít jiný chain - *ConversationalRetrievalChain*.

In [ ]:
from langchain.chains import ConversationalRetrievalChain

Parametry jsou (krom přidání paměti) stejné jako v předchozím chainu. Nicméně v pozadí přibyl další krok, ve kterém chain vezme historii i novou uživatelovu otázku a zkondenzuje je do otázky přežvýkané. Tento krok má následující podobu: 
```
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n\nHuman: Jak dlouho debentní karta platí?\nAssistant: Debetní karta platí do posledního dne měsíce a roku doby platnosti, která je uvedena na kartě.\nFollow Up Input: Jak se to liší v případě, kdy máme kreditní kartu?\nStandalone question:"
  ]
}
```
díky čemuž vznikne otázka
```
"Jak dlouho platí kreditní karta?"
```

In [ ]:
memory_chain = ConversationalRetrievalChain.from_llm(
    llm=chat,
    retriever=vectordb.as_retriever(),
    memory=conv_buff_memory
)

Pozn.: bacha, slovník vstupující do provolávání chainu musí mít klíč "question".

In [ ]:
question = "Jak dlouho debentní karta platí?"
result = memory_chain({"question": question})

In [ ]:
result

In [ ]:
question_2 = "Jak se to liší v případě, kdy máme kreditní kartu?"
result = memory_chain({"question": question_2})

In [ ]:
result

BTW odpověď je asi správně, ale v celém dokumentu o kreditních kartách nebyla ani zmínka.  
Pozn.: pokud chceme vidět zdrojové dokumenty, musíme vložit do konstruktoru *ConversationalRetrievalChain* parametr *verbose* s hodnotou True.

#### Automatické používání metadat

Načtěme si nejprve krom pdfka o debentních kartách i pdfko o kartách kreditních.

In [ ]:
loaders_list = [
    PyPDFLoader("source_files\\podminky_debetnich_karet.pdf"),
    PyPDFLoader("source_files\\Podminky-osobnich-kreditnich-karet.pdf")
]

pages = []
for loader in loaders_list:
    pages = pages + loader.load()

Nyní postupujme obvyklým způsobem (bez metadat).

In [ ]:
chunk_size = 250
chunk_overlap = 50

rec_text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
)

rec_splits = rec_text_splitter.split_documents(pages)

In [ ]:
embedding = OpenAIEmbeddings()
simple_doc_dir = "embeddings\\two_cards\\"

In [ ]:
vectordb = Chroma.from_documents(
    documents=rec_splits,
    embedding=embedding,
    persist_directory=simple_doc_dir
)
vectordb.persist()

In [ ]:
vectordb = Chroma(persist_directory=simple_doc_dir, embedding_function=embedding)

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Answer in Czech language. 
{context}
Question: {question}
Helpful Answer:"""
qa_chain_prompt = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_chain_prompt}
)

In [ ]:
question = "Jak je to s aktivováním debentní karty?"
result = qa_chain({"query": question})

Pozn.: prosím o ignorování pole "topic" a "product" v metadatech - jedná se o produkt následujících kroků. zde jsou irrelevantní.

In [ ]:
result

Všimněte si, že i když byla odpověď asi správná, dostal se mezi nejvhodnější fragmenty i jeden fragment z pdfka o kartách kreditních.  
Jaké je řešení? Inu, lze použít metadata. Pokud člověk pracuje s dokumenty s lehce zpracovatelným názvem (typu "podminky_produkt_1.pdf", "podminky_produkt_2.pdf") a má štěstí, nemusí přidávat žádná nová metadata. I v našem poměrně primitivním případě to ale moc nefungovalo. 
Musíme tedy data ve vektorové databázi obohatit o nové metadatové pole.

In [ ]:
debit_cards_ids = vectordb.get(where={"source": "source_files\\podminky_debetnich_karet.pdf"})["ids"]
debit_cards_metadatas = vectordb.get(where={"source": "source_files\\podminky_debetnich_karet.pdf"})["metadatas"]

In [ ]:
credit_cards_ids = vectordb.get(where={"source": "source_files\\Podminky-osobnich-kreditnich-karet.pdf"})["ids"]
credit_cards_metadatas = vectordb.get(where={"source": "source_files\\Podminky-osobnich-kreditnich-karet.pdf"})["metadatas"]

Hned tady může být zádrhel, když do metadat vložíme slova ve tvaru, které později jazykový model nevygeneruje. Metadatové filtrování totiž není fuzzy, ale přesné. Tudíž když modle vyvodí, že se v otázce "Co musím udělat pro blokaci produktu debentní karty?" ptáme na produkt "debentní karta", nemůžeme mít v metadatech "debentní kart**y**". Podobně bacha na case sensitivitu - model (přesněji vláček modelu a databázového enginu) začne pro otázku "Co musím udělat pro blokaci produktu Debentní karta?" hledat produkt "Debentní karta" a též nic nenajde.

In [ ]:
for one_metadata in debit_cards_metadatas:
    one_metadata["product"] = "debentní karta"

In [ ]:
for one_metadata in credit_cards_metadatas:
    one_metadata["product"] = "kreditní karta"

Provedeme update a změny databáze uložíme.

In [ ]:
vectordb._collection.update(ids=debit_cards_ids, metadatas=debit_cards_metadatas)

In [ ]:
vectordb._collection.update(ids=credit_cards_ids, metadatas=credit_cards_metadatas)

In [ ]:
vectordb.persist()

Pro získání fragmentů dokumentů, u kterých už budou hrát roli metadata, musíme použít *SelfQueryRetriever*. Ten vyžaduje mít nainstalovaný balíček [lark](https://pypi.org/project/lark/).

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

Musíme mu mimo jiné podhodit popis metadat.

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="product",
        description="Product name",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="The document source",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the document",
        type="integer",
    ),
]

Též do *SelfQueryRetriever* vložíme parametr *document_contents*.

In [ ]:
document_content_description = "Bank products"
retriever = SelfQueryRetriever.from_llm(
    llm=chat,
    vectorstore=vectordb,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True
)

No a když jsem tohle všechno udělal, tak... to stejně nefungovalo :D. Možná jsem se špatně dotazoval, možná tenhle typ úlohy funguje snáze v angličtině. Zkusil jsem tak editovat soubor {jméno_vašeho_environmentu}\Lib\site-packages\langchain\chains\query_constructor\prompt.py, ve kterém je šablona za tvoření queriny do Chromy zodpovědná. No, snad jsem přitom nic nerozbil...   
Změněné byly tyto části souboru:
```
BANK_DATA_SOURCE = """\
```json
{
    "content": "Informace k bankovnímu produktu",
    "attributes": {
        "produkt": {
            "type": "string",
            "description": "Název produktu"
        },
        "zamereni": {
            "type": "string",
            "description": "Zaměření dokumentu, jedna z možností \"ceník\", \"všeobecné informace\""
        }
    }
}
```\
""".replace(
    "{", "{{"
).replace(
    "}", "}}"
)

FULL_ANSWER = """\
```json
{{
    "query": "cena spotřebitelský úvěr",
    "filter": "and(eq(\\"produkt\\", \\"spotřebitelský úvěr\\"), \
eq(\\"zamereni\\", \\"ceník\\"))"
}}
```\
"""

DEFAULT_EXAMPLES = [
    {
        "i": 1,
        "data_source": BANK_DATA_SOURCE,
        "user_query": "Kolik stojí vyřízení spotřebitelského úvěru",
        "structured_request": FULL_ANSWER,
    },
    {
        "i": 2,
        "data_source": BANK_DATA_SOURCE,
        "user_query": "V kolik jede vlak?",
        "structured_request": NO_FILTER_ANSWER,
    },
]

EXAMPLES_WITH_LIMIT = [
    {
        "i": 1,
        "data_source": BANK_DATA_SOURCE,
        "user_query": "Kolik stojí vyřízení spotřebitelského úvěru",
        "structured_request": FULL_ANSWER,
    },
    {
        "i": 2,
        "data_source": BANK_DATA_SOURCE,
        "user_query": "V kolik jede vlak",
        "structured_request": NO_FILTER_ANSWER,
    },
    {
        "i": 3,
        "data_source": BANK_DATA_SOURCE,
        "user_query": "What are three songs about love",
        "structured_request": WITH_LIMIT_ANSWER,
    },
]
```

Pro dotaz "Co musím udělat pro blokaci debentní karty?" (popř. "Co musím udělat pro blokaci debentní karty od KB?") se vygeneruje
```
"```json\n{\n    \"query\": \"blokace debentní karta\",\n    \"filter\": \"NO_FILTER\"\n}\n```"
```
Tj. querina správná, ale filtr se neuplatnil.
Při dotazu "Co musím udělat pro blokaci produktu debentní karta?" je už filtr správný
```
"```json\n{\n    \"query\": \"blokace debentní karta\",\n    \"filter\": \"eq(\\\"product\\\", \\\"debentní karta\\\")\"\n}\n```"
```
Ale popravdě nedokážu si představit, že by v praxi uživatel používal takto pro model nápovědný dotaz.  
Pro zajímavost - na dotaz "Kolik budu platit u pojištění Merlin od KB?" by vznikl filtr
```
"```json\n{\n    \"query\": \"platit pojištění Merlin KB\",\n    \"filter\": \"and(eq(\\\"product\\\", \\\"Merlin\\\"), eq(\\\"source\\\", \\\"KB\\\"))\"\n}\n```"
```
Asi by to chtělo ten šablonový prompt ze souboru trochu vylepšit.

In [ ]:
question = "Co musím udělat pro blokaci produktu debentní karta?"
result = retriever.get_relevant_documents(question)

In [ ]:
result

#### Prefix před fragmenty
Co se týče problému nalezení fragmentu ke správné problematice, lze na internetu nalézt doporučení, aby se jméno problematiky vložilo na začátek každého fragmentu.

In [ ]:
chunk_size = 250
chunk_overlap = 50

rec_text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
)

loader_1 = PyPDFLoader("source_files\\podminky_debetnich_karet.pdf")
pages_1 = loader_1.load()
rec_splits_1 = rec_text_splitter.split_documents(pages_1)
for one_doc in rec_splits_1:
    one_doc.page_content = "debentní karta \t " + one_doc.page_content


loader_2 = PyPDFLoader("source_files\\Podminky-osobnich-kreditnich-karet.pdf")
pages_2 = loader_2.load()
rec_splits_2 = rec_text_splitter.split_documents(pages_2)
for one_doc in rec_splits_2:
    one_doc.page_content = "kreditní karta \t " + one_doc.page_content

all_splits = rec_splits_1 + rec_splits_2

In [ ]:
embedding = OpenAIEmbeddings()
doc_dir = "embeddings\\fragment_prefix\\"

vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding,
    persist_directory=doc_dir
)
vectordb.persist()

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Answer in Czech language. 
{context}
Question: {question}
Helpful Answer:"""
qa_chain_prompt = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_chain_prompt}
)

Zdá se, že funkčnost takového přístupu je omezená, nicméně pro úplné zatracení či naopak vychválení do nebes by to chtělo větší otestování.

In [ ]:
result = qa_chain({"query": "Co musím udělat pro blokaci debentní karty?"})
result

In [ ]:
result = qa_chain({"query": "Co dělat s kreditkou zaseklou v bankomatu?"})
result

In [ ]:
result = qa_chain({"query": "Co dělat s debetní kartou zaseklou v bankomatu?"})
result

### Kompletní ukázky
V textu výše jsem se snažil komentovat všechny možné věci, které je třeba při vytvoření langchainové aplikace vzít v úvahu. Může být ale užitečné vidět vše pohromadě a právě proto existuje tato kapitola.
#### Obyčejný chatbot

In [ ]:
import os
import dotenv
import openai

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.memory import ConversationBufferWindowMemory 
from langchain.chains import LLMChain

dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]


chat = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are Sauron from 'Lord of the Ring' novel."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
window_memory = ConversationBufferWindowMemory(k=5, memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
    memory=window_memory
)

In [ ]:
conversation.invoke(input="Hi, who are you?")

In [ ]:
conversation.invoke(input="I have One ring, but I will never give it to you!")

#### Q&A chatbot
Skript načítací záznamy do databáze se nachází níže. Možná někoho zarazí metoda listu *extend*. Ta na konec svého mateřského listu vloží všechny elementy iterable objektu (např. listu) ze svého argumentu. Tj. například pro mateřský list [1, 2, 3] a dodatečný list [4, 5] vede extend na [1, 2, 3, 4, 5], zatímco append by skončil s [1, 2, 3, [4, 5]]. No a toto je chtění chování, když si uvědomíme, že *loader.load()* nám dá list Document objektů, které odpovídají jednotlivým stránkám jednoho pdfka.

In [ ]:
import os
import dotenv
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

loaders = [
    PyPDFLoader("source_files\\penzijni-plan-1-1-16.pdf"),
    PyPDFLoader("source_files\\penzijni-plan-3-1-16.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

embedding = OpenAIEmbeddings()

chunk_size = 1000
chunk_overlap = 100

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["ČLÁNEK", "\n\n", "\n", "."]
)

splits = r_splitter.split_documents(docs)

persist_directory = "embeddings\\chroma_persist_dir\\"

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

vectordb.persist()

Skript realizující dotazování

In [ ]:
import os
import dotenv
import openai
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

persist_directory = "embeddings\\chroma_persist_dir\\"

embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible. Answer in Czech language.
{context}
Question: {question}
Helpful answer:"""

chat = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo") 

memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    return_message=True,
    output_key="answer"
)

qa_chain_prompt = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_chain_prompt}
)

while True:
    question = input("Napiš otázku (exit pro opuštění chatu):\n")
    if question == "exit":
        break 
    result = qa_chain.invoke({"query":question})
    print(result["result"])
    print()
    print("Zdrojové dokumenty:")
    for one_source_doc in result["source_documents"]:
        print("--------------------------")
        print("    ", one_source_doc)
        print("--------------------------")

## TO DO

stuff vs map-reduce vs postupné zpracování